In [37]:
from __future__ import print_function
from traitlets.utils.descriptions import describe
import six
import types
import inspect

#### Decorators

In [2]:
def my_stringify(my_function):
    print('my_function', my_function)
    print('entered my_stringify')
    def my_wrapper(*args, **kwargs):
        print('enter my_wrapper')
        result = my_function(*args, **kwargs)
        print('result computed')
        return str(result)
    print('return my_wrapper')
    return my_wrapper

In [3]:
def my_result(my_function):
    print('my_function', my_function)
    print('entered my_result')
    def my_wrapper(*args, **kwargs):
        print('entered my_wrapper')
        result = my_function(*args, **kwargs)
        print('result computed')
        return result
    print('return my_wrapper')
    return my_wrapper

In [4]:
@my_stringify
def my_square_function(x):
    return x**2

my_function <function my_square_function at 0x1096011e0>
entered my_stringify
return my_wrapper


In [5]:
my_square_function(2)

enter my_wrapper
result computed


'4'

In [6]:
@my_result
def my_cube_function(x):
    return x**3

my_function <function my_cube_function at 0x109601268>
entered my_result
return my_wrapper


In [7]:
my_cube_function(3)

entered my_wrapper
result computed


27

#### Descriptors

In [8]:
class my_descriptor(object):
    
    def __set__(self, obj, value):
        print("setting %r to %r" % (value, obj))

    def __get__(self, obj, cls):
        print("getting from %r" % obj)

    def __delete__(self, obj):
        print("deleting from %r" % obj)

In [9]:
class my_class(object):
    my_x = my_descriptor()

In [10]:
my_object = my_class()

In [11]:
my_object.my_x

getting from <__main__.my_class object at 0x1096240f0>


In [12]:
my_object.my_x = 5

setting 5 to <__main__.my_class object at 0x1096240f0>


In [13]:
my_object.my_x = 6

setting 6 to <__main__.my_class object at 0x1096240f0>


In [14]:
my_object.my_x = 'a value'

setting 'a value' to <__main__.my_class object at 0x1096240f0>


In [15]:
del my_object.my_x

deleting from <__main__.my_class object at 0x1096240f0>


#### Metaclasses

In [36]:
for cls in (int, str, bool, object, type):
    print("It is %s that %r is an instance of %r" % (isinstance(cls, type), cls, type))

It is True that <class 'int'> is an instance of <class 'type'>
It is True that <class 'str'> is an instance of <class 'type'>
It is True that <class 'bool'> is an instance of <class 'type'>
It is True that <class 'object'> is an instance of <class 'type'>
It is True that <class 'type'> is an instance of <class 'type'>


In [17]:
class Foobar:
    pass

print('type(Foobar):', type(Foobar))
foo = Foobar()
print('foo: ', foo)
print('type(foo): ', foo)
print('isinstance(foo, Foobar): ', isinstance(foo, Foobar))
#idk why this statement is false
print('isinstance(Foobar, type):', isinstance(Foobar, type))

type(Foobar): <class 'type'>
foo:  <__main__.Foobar object at 0x1096272b0>
type(foo):  <__main__.Foobar object at 0x1096272b0>
isinstance(foo, Foobar):  True
isinstance(Foobar, type): True


#### Traitlets

In [21]:

"""
__dict__: The namespace supporting arbitrary function attributes.

A module object has a namespace implemented by a dictionary object 
(this is the dictionary referenced by the __globals__ attribute of 
functions defined in the module). Attribute references are translated 
to lookups in this dictionary, e.g., m.x is equivalent to m.__dict__["x"].

Attribute assignment updates the module’s namespace 
dictionary, e.g., m.x = 1 is equivalent to m.__dict__["x"] = 1.

Special read-only attribute: __dict__ is the module’s 
namespace as a dictionary object.
"""
class Feature(object):
    
    klass = None
    
    def class_init(self, cls, name):
        print("the feature %r was assigned to %r" % (self, name))
        self.this_class = cls
        self.name = name

    def __set__(self, obj, value):
        if isinstance(value, self.klass):
            obj.__dict__[self.name] = value
        else:
            raise TypeError("Expected a %r instance, not %s" % (self.klass, describe("the", value)))
    
    def __get__(self, obj, cls):
        if obj is None:
            # this is true when accessing the attributed
            # from the class, not instances of the class
            return self
        return obj.__dict__[self.name]
    
    def __delete__(self, obj):
        del obj.__dict__[self.name]



In [22]:
class MetaHasFeatures(type):

    def __init__(cls, name, bases, classdict):
        print("creating %r" % cls)
        for k, v in classdict.items():
            if isinstance(v, Feature):
                v.class_init(cls, k)


In [23]:
class HasFeatures(six.with_metaclass(MetaHasFeatures, object)):
    pass

creating <class '__main__.HasFeatures'>


In [24]:
"""
MyInt inherits Feature
klass is overriden -> only accepts ints
"""
class MyInt(Feature):
    # this feature only accepts ints
    klass = int

In [25]:
"""
MyStr inherits Feature
klass is overriden -> only accepts ints
"""
class MyStr(Feature):
    # this feature only accepts strings
    klass = str

In [33]:
class MyBool(Feature):
    # this feature only accepts boolean
    klass = bool

In [41]:
"""
MyClass inherits HasFeatures
six.with_metaclass(metaclass, *bases)
Create a new class with base classes bases and metaclass metaclass. 
This is designed to be used in class declarations
"""
class MyClass(HasFeatures):
    i = MyInt()
    s = MyStr()
    b = MyBool()
mc = MyClass()

creating <class '__main__.MyClass'>
the feature <__main__.MyInt object at 0x109974b70> was assigned to 'i'
the feature <__main__.MyStr object at 0x109974e80> was assigned to 's'
the feature <__main__.MyBool object at 0x109974e10> was assigned to 'b'


In [38]:


# try:
#     mc.i = 1
# except Exception as e:
#     print(e)

# try:
#     mc.s = "i = 1"
# except Exception as e:
#     print(e)
    
# try:
#     mc.b = True
# except Exception as e:
#     print(e)


-------------------------------
assigned the value 1
-------------------------------
assigned the value i = 1
-------------------------------
assigned the value True
-------------------------------
get 1 from <__main__.MyClass object at 0x109627630>
-------------------------------
get i = 1 from <__main__.MyClass object at 0x109627630>
-------------------------------
get True from <__main__.MyClass object at 0x109627630>


In [42]:
print("-------------------------------")
mc.i = 1
print("assigned the value %r" % 1)

print("-------------------------------")
mc.s = "i = 1"
print("assigned the value %s" % "i = 1")

print("-------------------------------")
mc.b = True
print("assigned the value %r" % True)

-------------------------------
assigned the value 1
-------------------------------
assigned the value i = 1
-------------------------------
assigned the value True


In [43]:
print("-------------------------------")
print("get %r from %r" % (mc.i, mc))
print("-------------------------------")
print("get %s from %r" % (mc.s, mc))
print("-------------------------------")
print("get %r from %r" % (mc.b, mc))

-------------------------------
get 1 from <__main__.MyClass object at 0x109974f28>
-------------------------------
get i = 1 from <__main__.MyClass object at 0x109974f28>
-------------------------------
get True from <__main__.MyClass object at 0x109974f28>


#### Let's go back to decorators...

A `BaseDecorator` object. When called, it must still act like a method.

In [50]:
class BaseDecorator(object):

    def __init__(self, name, method):
        print('__init__')
        self.feature_name = name
        self.method = method
    
    def __call__(self, *args, **kwargs):
        print('__call__')
        return self.method(*args, **kwargs)
    
    def __get__(self, obj, cls):
        print('__get__')
        if obj is None:
            return self
        return types.MethodType(self.method, obj)

An `observe` decorator. Wraps a method in an `Observer` object.

In [52]:
def observe(name):
    print('observe decorator')
    def setup(method):
        print('observe decorator setup')
        return Observer(name, method)
    return setup

An `Observer` object. After the instance is created it must register itself as an observer.

In [54]:
class Observer(BaseDecorator):
    print('class Observer')
    def instance_init(self, obj):
        print('instance_init')
        obj.observe(self, self.feature_name)
        print('obj.observe')

class Observer


An `OtherFeature` descriptor. It will notify its owner whenever a value changes

In [48]:
class OtherFeature(Feature):
    
    def __set__(self, obj, new):
        old = obj.__dict__.get(self.name, None)
        super(OtherFeature, self).__set__(obj, new)
        if new != old:
            data = dict(old=old, new=new, name=self.name, owner=obj)
            obj.notify(data)

A new `HasOtherFeatures` class. 

It will find `BaseObservers` and call their `instance_init` methods so they can register themselves Offer a method `observe` that will register observers Offer a `notify` which will be used to pass change data to observers.

In [49]:
class HasOtherFeatures(HasFeatures):

    def __init__(self):
        self._observers = {}
        self.setup()
    
    def setup(self):
        for k, v in inspect.getmembers(type(self)):
            if isinstance(v, BaseDecorator):
                v.instance_init(self)
    
    def observe(self, handler, name):
        if name in self._observers:
            olist = self._observers[name]
        else:
            olist = []
            self._observers[name] = olist
        if handler not in olist:
            olist.append(handler)
    
    def notify(self, data):
        for o in self._observers.get(data["name"], []):
            if isinstance(o, BaseDecorator):
                o(self, data)
            else:
                o(data)

creating <class '__main__.HasOtherFeatures'>
